In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
import re

In [2]:
import gensim
from gensim import corpora, models, similarities

import nltk
from nltk.corpus import stopwords

In [3]:
pip install lightfm

  Using cached lightfm-1.17.tar.gz (316 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for lightfm
Failed to build lightfm
  Running setup.py install for lightfm: started
  Running setup.py install for lightfm: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [25 lines of output]
  Compiling without OpenMP support.
  V:\PYTHON\lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
    warnings.warn(
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\lightfm
  copying lightfm\cross_validation.py -> build\lib.win-amd64-cpython-39\lightfm
  copying lightfm\data.py -> build\lib.win-amd64-cpython-39\lightfm
  copying lightfm\evaluation.py -> build\lib.win-amd64-cpython-39\lightfm
  copying lightfm\lightfm.py -> build\lib.win-amd64-cpython-39\lightfm
  copying lightfm\_lightfm_fast.py -> build\lib.win-amd64-cpython-39\lightfm
  copying lightfm\__init__.py -> build\lib.win-amd64-cpython-

In [4]:
from lightfm import LightFM 

ModuleNotFoundError: No module named 'lightfm'

In [5]:
import scipy
from scipy.spatial.distance import cdist
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from scipy.linalg import svd

In [6]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.model_selection import train_test_split

# Reading data sets

1. rated_articles - Contains user_interests with ratings
2. news_articles - Contains raw articles without user data amalgamation

## news_articles

In [7]:
data = pd.read_csv(r'0_news_articles.csv')
data.head()

,Article_id,Title,Description,Date,Category,URL
0,0,Fire at Vaishno Devi shrine complex; cash coun...,"No one was injured in the fire, which broke ou...","June 8, 2021 7:28:32 pm",India,https://indianexpress.com/article/india/vaishn...
1,1,"Had not gone to meet Nawaz Sharif, says Uddhav...",Uddhav Thackeray led a delegation of his cabin...,"June 8, 2021 6:56:40 pm",India,https://indianexpress.com/article/india/had-no...
2,2,Corruption case: Former Haryana I-T deputy com...,It was in 2016 that the CBI had arrested Nitin...,"June 8, 2021 6:25:24 pm",India,https://indianexpress.com/article/india/corrup...
3,3,Kannur MP K Sudhakaran appointed chief of Cong...,Sudhakaran will replace Ramachandran who had a...,"June 8, 2021 5:04:40 pm",India,https://indianexpress.com/article/india/sudhak...
4,4,"Kerala girl of Class 5 writes to CJI, lauds SC...",Chief Justice N V Ramana responded to the Clas...,"June 8, 2021 4:43:10 pm",India,https://indianexpress.com/article/india/kerala...


## Collaborative Filtering

**Need:** Ratings Matrix so I generated user profile with ratings

In [8]:
rating = pd.read_csv(r'3_rated_articles.csv')
print(rating.shape)
rating.drop(columns= rating.columns[0], 
        axis=1, 
        inplace=True)
rating.head()

(1100, 8)


,Article_id,Title,UserId,SessionId,Article Rank,Time Spent (seconds),Ratings
0,1,"Had not gone to meet Nawaz Sharif, says Uddhav...",1,1,2,24,3
1,2,Corruption case: Former Haryana I-T deputy com...,1,1,3,18,2
2,6,Uddhav Thackeray meets PM Modi; discusses Mara...,1,1,7,47,3
3,7,"New Covid-19 vaccination guidelines out, alloc...",1,1,8,47,4
4,9,Fire at TMC MLA Madan Mitra’s residence in Kol...,1,1,10,73,4


In [9]:
rating.tail()

,Article_id,Title,UserId,SessionId,Article Rank,Time Spent (seconds),Ratings
1095,2237,"‘They are mine to look after, to care, to love...",2227,2222,8,36,5
1096,2238,FSSAI recommends including soy foods in your d...,2228,2223,9,98,5
1097,2241,‘Billie Eilish as we’ve never seen her before’...,2231,2226,2,17,3
1098,2243,Bhagyashree pens note on ‘prioritising one’s o...,2233,2228,4,34,4
1099,2249,‘Love wins’: Rita Wilson and Tom Hanks celebra...,2239,2234,10,73,1


In [10]:
n_users = int(rating.UserId.nunique())
n_article = int(rating.Article_id.nunique())
print("Number of users: " , n_users)
print("Number of articles: ", n_article)

Number of users:  1095
Number of articles:  1100


In [11]:
user_pivot = rating.pivot_table(index = 'UserId', columns = 'Article_id', values = 'Ratings')
user_pivot.head()

Article_id,1,2,6,7,9,11,13,14,15,16,...,2227,2228,2230,2231,2232,2237,2238,2241,2243,2249
UserId,,,,,,,,,,,,,,,,,,,,,
1,3.0,2.0,3.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
user_pivot.shape

(1095, 1100)

In [13]:
user_pivot = user_pivot.fillna(0)
user_pivot_matrix = user_pivot.values
user_pivot_matrix

array([[3., 2., 3., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 3., 0., 0.],
       [0., 0., 0., ..., 0., 4., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [14]:
user_item_pivot_sparse = csr_matrix(user_pivot)

In [15]:
n_factors = 150
U, sigma, Vt = svds(user_pivot_matrix, k = n_factors)

sigma = np.diag(sigma)
sigma.shape

(150, 150)

In [16]:
all_user_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_ratings_norm = (all_user_ratings - all_user_ratings.min()) / (all_user_ratings.max() - all_user_ratings.min())
all_user_ratings_norm

array([[0.71452067, 0.52425741, 0.71452067, ..., 0.14373089, 0.14373089,
        0.14373089],
       [0.14373089, 0.14373089, 0.14373089, ..., 0.14373089, 0.14373089,
        0.14373089],
       [0.14373089, 0.14373089, 0.14373089, ..., 0.14373089, 0.14373089,
        0.14373089],
       ...,
       [0.14373089, 0.14373089, 0.14373089, ..., 0.14373089, 0.14373089,
        0.14373089],
       [0.14373089, 0.14373089, 0.14373089, ..., 0.14373089, 0.1968651 ,
        0.14373089],
       [0.14373089, 0.14373089, 0.14373089, ..., 0.14373089, 0.14373089,
        0.14373089]])

In [17]:
cf_preds_df = pd.DataFrame(all_user_ratings_norm, columns = user_pivot.columns).transpose()

In [18]:
class Collaborative:
    
    name = "Collaborative Filter"
    
    def __init__(self, predictions, items = None):
        self.predictions = predictions
        self.items = items
        
    def get_model_name(self):
        return self.name
            
    def recommend_items(self, user_id, items_ignore = [], topn = 10, verbose = False):
        sorted_preds = self.predictions[user_id].sort_values(ascending = False).reset_index()

        recommendations = sorted_preds[~sorted_preds['Article_id'].isin(items_ignore)].head(topn)

        if verbose:
            if self.items is None:
                raise Exception('"items" required in verbose mode')

            recommendations = recommendations.merge(self.items, how = 'left', left_on = 'Article_id', 
                                                    right_on = 'Article_id')[['Article_id', 'Title']]


        return recommendations

In [19]:
model = Collaborative(cf_preds_df, data)

In [20]:
model.recommend_items(user_id = 224, verbose = True)

,Article_id,Title
0,2155,‘Going to breastfeed Aaravv for as long as I c...
1,243,Karnataka allows export-oriented units to open...
2,1,"Had not gone to meet Nawaz Sharif, says Uddhav..."
3,1422,The Specialist: Doubles coach Mathias Boe aimi...
4,1433,Cricketers’ Foundation begins initiative to he...
5,1431,Alisson-like header on overlap is on my bucket...
6,1428,"Belgium name De Bruyne, Lukaku, Hazard in 26-m..."
7,1427,‘We are grateful to BCCI’: CA’s Nick Hockley o...
8,1423,Ian Chappell backs Pat Cummins over Steve Smit...
9,1419,Italy coach Roberto Mancini gets contract exte...


## Hybrid Recommendor System

**Using:** LightRF

In [21]:
user_pivot.head()

Article_id,1,2,6,7,9,11,13,14,15,16,...,2227,2228,2230,2231,2232,2237,2238,2241,2243,2249
UserId,,,,,,,,,,,,,,,,,,,,,
1,3.0,2.0,3.0,4.0,4.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
u_id = list(user_pivot.index)
u_dict = {}
counter = 0

for i in u_id:
    u_dict[i] = counter
    counter += 1

In [23]:
len(u_dict)

1095

In [24]:
# convert to csr matrix
u_interaction_csr = csr_matrix(user_pivot.values)
u_interaction_csr

<1095x1100 sparse matrix of type '<class 'numpy.float64'>'
	with 1100 stored elements in Compressed Sparse Row format>

In [25]:
item_dict ={}
df = rating[['Article_id', 'Title']].sort_values('Article_id').reset_index()

for i in range(df.shape[0]):
    item_dict[(df.loc[i,'Article_id'])] = df.loc[i,'Title']

In [26]:
len(item_dict)

1100

### LightFM not working, so hopes low

In [27]:
model = LightFM(loss = 'warp', random_state = 2016, learning_rate = 0.90, no_components = 150, user_alpha = 0.000005)
model = model.fit(u_interaction_csr, epochs = 100, num_threads = 16, verbose = False)

NameError: name 'LightFM' is not defined

In [28]:
title = list(data['Title'])
title[:10]

['Fire at Vaishno Devi shrine complex; cash counter damaged',
 'Had not gone to meet Nawaz Sharif, says Uddhav Thackeray as he plays down one-on-one meeting with PM Modi',
 'Corruption case: Former Haryana I-T deputy commissioner gets 4 years in prison',
 'Kannur MP K Sudhakaran appointed chief of Congress in Kerala',
 'Kerala girl of Class 5 writes to CJI, lauds SC for saving lives in fight with Covid',
 'Madhya Pradesh govt gets HC notice on communal clashes during fundraising for Ram temple',
 'Uddhav Thackeray meets PM Modi; discusses Maratha quota issue, GST compensation',
 'New Covid-19 vaccination guidelines out, allocation based on state population',
 'Dantewada: 24-year-old tribal woman killed in ‘maoist encounter’; family claims it was staged, alleges rape',
 'Fire at TMC MLA Madan Mitra’s residence in Kolkata']

In [29]:
total = data.isnull().sum().sort_values(ascending = False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
missing_data.head()

,Total,Percent
Article_id,0,0.0
Title,0,0.0
Description,0,0.0
Date,0,0.0
Category,0,0.0
